# PennyLane Building Blocks of a Quantum Neural Network.
This notebook explains the templates, layers and overview pf how to make a Quantum Model.

## Input -> Encode classical data
The quantum model input needs to be encoded using an embedding circuit. PennyLane has a few of these
* Basis: this is used for integer or binary
* Amplitude Encoding: Normalized to 2^n, pad if data is not in right form. Data Types Integer, float, complex. A sum of features must equal 1
* Displacement
* Angle Embedding: N features in n qubits with rotation angles. N<= n>

## Using Entangled circuits to create layers


In [ ]:
# Import useful packages

import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers

## Input styles

Basis Embedding

Amplitude Encoding: Integers/foat/complex

Angle Embedding: Integer/Float/Complex



In [ ]:
# BasisEmbedding
wires = 2
basis_dev = qml.device('default.qubit, wires)
@qml.qnode(basis_dev)
def basis_encoder(data):
    qml.BasisEmbedding(data, wires)
    return qml.state()

# Angle Embedding
angle_dev = qml.device('default.qubit, wires)
@qml.qnode(angle_dev)
def angle_encoder(data):
    qml.AngleEmbedding(features=data, wires=wires, rotation = 'X')
    return qml.state()

# Displacement Embedding
# Amplitude Encoding
amp_dev = qml.device('default.qubit, wires)
@qml.qnode(amp_dev)
def amp_encoder(data):
    qml.AmplitudeEmbedding(data, wires, pad_width= 0, normalize= True)
    return qml.state()

# QOADO


## Entangled templates
PennyLane has multiple templates for 

### Basic Entangled layers

### Random Layers

### CVNeural Net layers

### Strongly entangled layers

## Typical Hidden Layers

### Fully connected layer

In [ ]:
# 2 Qubit FCN layer for quantum applications
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

# Create weight shape
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}
# layer shape is (layers x qubits) so this example is 6 x 2

# Converting to layer
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

# Sequential model
clayer_1 = torch.nn.Linear(2, 2)
clayer_2 = torch.nn.Linear(2, 2)
softmax = torch.nn.Softmax(dim=1)
layers = [clayer_1, qlayer, clayer_2, softmax]
model = torch.nn.Sequential(*layers)

# General model
class HybridModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.clayer_1 = torch.nn.Linear(2, 4)
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_2 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.clayer_2 = torch.nn.Linear(4, 2)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.clayer_1(x)
        x_1, x_2 = torch.split(x, 2, dim=1)
        x_1 = self.qlayer_1(x_1)
        x_2 = self.qlayer_2(x_2)
        x = torch.cat([x_1, x_2], axis=1)
        x = self.clayer_2(x)
        return self.softmax(x)

model = HybridModel()